In [33]:
def classify(x):
    for i in range(len(x)):
        if len(x[i])==1:
            x[i] = 0
        else:
            x[i] = 1
    return x

In [34]:
def read_text_1(file_path):
    with open(file_path, "r", encoding="utf-8") as file:
        lines = file.readlines()

    for i in range(len(lines)):
        lines[i]=lines[i].strip()

    return lines

In [35]:
def read_text(file_path):
    with open(file_path, "r", encoding="utf-8") as file:
        lines = file.readlines()

    for i in range(len(lines)):
        lines[i]=lines[i].strip()
        y = lines[i].split(' ')
        lines[i] = classify(y)

    return lines

In [36]:
l = read_text("Static_Idioms_Tags.txt")
sen = read_text_1("Static_Idioms_Words.txt")

In [37]:
len(sen)

21891

In [38]:
from transformers import AutoTokenizer, DistilBertForTokenClassification
import torch

In [45]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model = DistilBertForTokenClassification.from_pretrained("distilbert-base-uncased")

inputs = tokenizer(
    "This is a task to tokenize idiomatic parts of sentences", add_special_tokens=False, return_tensors="pt"
)

with torch.no_grad():
    logits = model(**inputs).logits

predicted_token_class_ids = logits.argmax(-1)

# Note that tokens are classified rather then input words which means that
# there might be more predicted token classes than words.
# Multiple token classes might account for the same word
predicted_tokens_classes = [model.config.id2label[t.item()] for t in predicted_token_class_ids[0]]

labels = predicted_token_class_ids
loss = model(**inputs, labels=labels).loss

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForTokenClassification: ['vocab_projector.weight', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN t

In [46]:
inputs

{'input_ids': tensor([[ 2023,  2003,  1037,  4708,  2000, 19204,  4697,  8909, 18994, 12070,
          3033,  1997, 11746]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [47]:
input_ids = inputs["input_ids"]
input_ids.squeeze()

tensor([ 2023,  2003,  1037,  4708,  2000, 19204,  4697,  8909, 18994, 12070,
         3033,  1997, 11746])

In [50]:
tokens = tokenizer.convert_ids_to_tokens(input_ids.squeeze())

In [51]:
tokens

['this',
 'is',
 'a',
 'task',
 'to',
 'token',
 '##ize',
 'id',
 '##iom',
 '##atic',
 'parts',
 'of',
 'sentences']

In [52]:
decoded_string = tokenizer.decode([ 2023,  2003,  1037,  4708,  2000, 19204,  4697,  8909, 18994, 12070,3033,  1997, 11746])
print(decoded_string)

this is a task to tokenize idiomatic parts of sentences
